In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import scipy.integrate as spi
from tqdm import tqdm
from matplotlib.colors import LogNorm, Normalize
from matplotlib import gridspec
from matplotlib.patches import ConnectionPatch
import scipy
from scipy.integrate import solve_ivp, odeint
import pints
from time import time
from numpy.linalg import norm
import pints.plot
import scipy.io
import pyreadr
import pandas as pd
import rpy2
from scipy.optimize import root, fsolve
from SALib.sample import saltelli, sobol, fast_sampler
from SALib.analyze import sobol, fast
import SALib


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/homebrew/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/lib/python3.10/s

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
fetaldata = np.array([[1.538461538461538547e-01, 5.000000000000000000e-01, 2.385589094449853953e-02, 4.868549172346640875e-04, 2.409931840311587137e-01, 3.894839337877312700e-03, 7.692307692307692735e-02], 
                      [9.357495881383855185e-02, 1.545304777594728174e-01, 4.250411861614497577e-01, 2.965403624382207531e-03, 2.859967051070840260e-01, 8.566721581548600306e-03, 2.932454695222405303e-02],
                      [6.140350877192982462e-03, 0.000000000000000000e+00, 2.631578947368420993e-03, 5.403508771929824706e-01, 1.140350877192982532e-02, 4.140350877192982559e-01, 2.543859649122806946e-02],
                      [3.707136237256719244e-03, 1.853568118628359622e-03, 2.780352177942539433e-03, 1.909175162187210506e-01, 5.560704355885078866e-03, 7.859128822984244866e-01, 9.267840593141797242e-03],
                      [1.089324618736383548e-03, 1.089324618736383548e-03, 1.089324618736383548e-03, 1.427015250544662328e-01, 6.535947712418300852e-03, 8.453159041394335826e-01, 2.178649237472767095e-03]])

t = np.linspace(0,52,100)
delta = 0.005
initmalignancy = np.zeros(7)
initmalignancy[0] = delta
gamma = 5e-3
gamma = 5e-3
X0 = np.hstack((fetaldata[0,:], initmalignancy))
k22 = 0
k33 = 0
k44 = 0
k55 = 0
k66 = 0

: 

In [ ]:
fetalt = [0, 1, 4, 7, 10]
bridge_rate = -scipy.optimize.curve_fit(lambda t,a,b: a*np.exp(b*t),
                                       fetalt,
                                       fetaldata[:,0])[0][1]

: 

In [ ]:
def RHS(t, X, parameters):
    # Set key simulation parameters 
    k11, k23, k25, k34, k56, k57, k77, gamma, mu  = parameters
    # Split up into proportions and malignancies
    props = X[0:7]
    malignancies = X[7:14]
    # RHS for the proportions
    prolif = np.array([k11, k22, k33, -k44, k55, 0, 0])
    M = np.array([[-bridge_rate, 0,0,0,0,0,0],
                   [bridge_rate, -(k23+k25), 0 ,0 , 0 ,0 ,0],
                   [0, k23,  -k34, 0,0,0,0],
                   [0, 0, k34, 0, 0, 0, 0],
                   [0, k25, 0, 0, -(k57 + k56), 0, 2*k77],
                   [0, 0, 0, 0, k56, 0, 0],
                   [0, 0, 0, 0, k57, 0, -k77]])

    RHS_props = prolif*props + M.dot(props) - props*(np.matmul(np.tile(prolif,(7,1)),props)+np.ones(7)*(k77*props[-1]))
    # RHS for the malignancies
    pc = np.tile(props,(7,1))
    mc = np.tile(malignancies,(7,1))
    prop_matrix = (mc.T-mc)*(pc.T/pc)  
    RHS_malignancies = (gamma+mu)*np.ones(7)*np.maximum(prolif,0)-(gamma+2*mu)*malignancies*np.ones(7)*np.maximum(prolif,0) - prop_matrix.dot(malignancies)- malignancies*np.ones(7)*(k77*props[-1])/props
    return np.hstack((RHS_props, RHS_malignancies))

def varyparamsplot(f, par, gamma, mu):
    t = np.linspace(0,52, 100)
    pars = np.hstack((par, [gamma,mu]))
    # Simulate with fixed initial condition
    solution = scipy.integrate.solve_ivp(RHS,
                                         t_span = (t[0],t[-1]), 
                                         y0 = X0, 
                                         method='LSODA', 
                                         t_eval=t, 
                                         args = [pars])['y']
    return solution


: 

In [ ]:
optMYCN = np.array([6.01795686e-01, 5.08273383e-03, 3.38001800e-01, 2.03309353e-02,
       1.00262147e-02, 6.68414312e-03, 1.47351638e-18])

optTERT = np.array([ 5.86392418e-01,  1.23191684e-02,  2.73485540e-01,  1.97106695e-02,
        2.39813146e-02,  7.55575665e-03, -1.80870775e-18])

optHIGH = np.array([ 6.58748040e-01,  1.75175457e-01,  1.54566579e-01,  8.95504785e-02,
        7.31058145e-02,  1.00259403e-01, -9.54290604e-18])
c_nc_ratio = fetaldata[-1,-1]/fetaldata[-1,-2]
initguess = [0, 1.80368429, 0.00395508,0.79357181,0.49522108, c_nc_ratio, 1.6706082 ]

: 

In [ ]:
val = 1e-1
solution = varyparamsplot(RHS, initguess, val, val)

: 

In [ ]:
print(sum([solution[i,:]*solution[i+7,:] for i in range(7)]))

: 

In [ ]:
problem = {
    'num_vars': 10,
    'names': ['k11', 'k23', 'k25', 'k34', 'k44', 'k56', 'k77', 'gamma', 'mu', 'dummy'],
    'bounds': [[-3, 0],
               [-3, 0],
               [-2, 0],
               [-3, 0],
               [-3, 0],
               [-3, 0],
               [-19, -17],
               [-4, -1],
               [-4, -1],
               [1, 1000]]
}

vals = SALib.sample.sobol.sample(problem, 16)
Y = np.zeros(len(vals))

: 